Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

# Inference PyTorch Bert Model with ONNX Runtime on GPU

In this tutorial, you'll learn how to load a Bert model from PyTorch, convert it to ONNX, and inference it for high performance using ONNX Runtime and NVIDIA GPU. In the following sections, we are going to use the Bert model trained with Stanford Question Answering Dataset (SQuAD) dataset as an example. Bert SQuAD model is used in question answering scenarios, where the answer to every question is a segment of text from the corresponding reading passage, or the question might be unanswerable.

This notebook is for GPU inference. For CPU inference, please look at another notebook [Inference PyTorch Bert Model with ONNX Runtime on CPU](PyTorch_Bert-Squad_OnnxRuntime_CPU.ipynb).

## 0. Prerequisites ##
It requires your machine to have a GPU, and a python environment with [PyTorch](https://pytorch.org/) installed before running this notebook.

#### GPU Environment Setup using AnaConda

First, we install [AnaConda](https://www.anaconda.com/distribution/) in a target machine and open an AnaConda prompt window when it is done. Then run the following commands to create a conda environment. This notebook is tested with PyTorch 1.5.0 and OnnxRuntime 1.3.0.

```console
conda create -n gpu_env python=3.7
conda activate gpu_env
conda install pytorch torchvision cudatoolkit=10.1 -c pytorch
conda install -c anaconda ipykernel
conda install -c conda-forge ipywidgets
python -m ipykernel install --user --name=gpu_env_py37
jupyter notebook
```
Finally, launch Jupyter Notebook and you can choose gpu_env_py37 as kernel to run this notebook.

Onnxruntime-gpu need specified version of CUDA and cuDNN. You can find the corresponding version in [requirements](https://github.com/microsoft/onnxruntime/tree/rel-1.3.0#system-requirements). If the version is different from above cudatoolkit version, you have to install them separately, and add their bin directories to PATH environment variable (See [CUDA and cuDNN Path](#CUDA-and-cuDNN-Path) below).

In [1]:
import sys
!{sys.executable} -m pip uninstall --quiet --yes onnxruntime-gpu
!{sys.executable} -m pip install --quiet onnxruntime-gpu
!{sys.executable} -m pip install --quiet --upgrade transformers
!{sys.executable} -m pip install --quiet --upgrade onnxconverter_common
!{sys.executable} -m pip install --quiet --upgrade onnxruntime-tools
!{sys.executable} -m pip install --quiet wget netron pandas

## 1. Load Pretrained Bert model ##

We begin by downloading the SQuAD data file and store them in the specified location. 

In [2]:
import os

cache_dir = "./squad"
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)

predict_file_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json"
predict_file = os.path.join(cache_dir, "dev-v1.1.json")
if not os.path.exists(predict_file):
    import wget
    print("Start downloading predict file.")
    wget.download(predict_file_url, predict_file)
    print("Predict file downloaded.")

Let's first define some constant variables.

In [3]:
# Whether allow overwriting existing ONNX model and download the latest script from GitHub
enable_overwrite = True

# Total samples to inference, so that we can get average latency
total_samples = 1000

# ONNX opset version
opset_version=11

Specify some model configuration variables.

In [4]:
# For fine-tuned large model, the model name is "bert-large-uncased-whole-word-masking-finetuned-squad". Here we use bert-base for demo.
model_name_or_path = "bert-base-cased"
max_seq_length = 128
doc_stride = 128
max_query_length = 64

Start to load model from pretrained. This step could take a few minutes. 

In [5]:
# The following code is adapted from HuggingFace transformers
# https://github.com/huggingface/transformers/blob/master/examples/run_squad.py

from transformers import (BertConfig, BertForQuestionAnswering, BertTokenizer)

# Load pretrained model and tokenizer
config_class, model_class, tokenizer_class = (BertConfig, BertForQuestionAnswering, BertTokenizer)
config = config_class.from_pretrained(model_name_or_path, cache_dir=cache_dir)
tokenizer = tokenizer_class.from_pretrained(model_name_or_path, do_lower_case=True, cache_dir=cache_dir)
model = model_class.from_pretrained(model_name_or_path,
                                    from_tf=False,
                                    config=config,
                                    cache_dir=cache_dir)
# load some examples
from transformers.data.processors.squad import SquadV1Processor

processor = SquadV1Processor()
examples = processor.get_dev_examples(None, filename=predict_file)

from transformers import squad_convert_examples_to_features
features, dataset = squad_convert_examples_to_features( 
            examples=examples[:total_samples], # convert enough examples for this notebook
            tokenizer=tokenizer,
            max_seq_length=max_seq_length,
            doc_stride=doc_stride,
            max_query_length=max_query_length,
            is_training=False,
            return_dataset='pt'
        )

add example index and unique id: 100%|██████████| 1000/1000 [00:00<00:00, 161306.98it/s]


## 2. Export the loaded model ##
Once the model is loaded, we can export the loaded PyTorch model to ONNX.

In [6]:
output_dir = "./onnx"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)   
export_model_path = os.path.join(output_dir, 'bert-base-cased-squad_opset{}.onnx'.format(opset_version))

import torch
use_gpu = torch.cuda.is_available()
device = torch.device("cuda" if use_gpu else "cpu")

# Get the first example data to run the model and export it to ONNX
data = dataset[0]
inputs = {
    'input_ids':      data[0].to(device).reshape(1, max_seq_length),
    'attention_mask': data[1].to(device).reshape(1, max_seq_length),
    'token_type_ids': data[2].to(device).reshape(1, max_seq_length)
}

# Set model to inference mode, which is required before exporting the model because some operators behave differently in 
# inference and training mode.
model.eval()
model.to(device)

if enable_overwrite or not os.path.exists(export_model_path):
    with torch.no_grad():
        symbolic_names = {0: 'batch_size', 1: 'max_seq_len'}
        torch.onnx.export(model,                                            # model being run
                          args=tuple(inputs.values()),                      # model input (or a tuple for multiple inputs)
                          f=export_model_path,                              # where to save the model (can be a file or file-like object)
                          opset_version=opset_version,                      # the ONNX version to export the model to
                          do_constant_folding=True,                         # whether to execute constant folding for optimization
                          input_names=['input_ids',                         # the model's input names
                                       'input_mask', 
                                       'segment_ids'],
                          output_names=['start', 'end'],                    # the model's output names
                          dynamic_axes={'input_ids': symbolic_names,        # variable length axes
                                        'input_mask' : symbolic_names,
                                        'segment_ids' : symbolic_names,
                                        'start' : symbolic_names,
                                        'end' : symbolic_names})
        print("Model exported at ", export_model_path)

Model exported at  ./onnx/bert-base-cased-squad_opset11.onnx


## 3. PyTorch Inference ##
Use PyTorch to evaluate an example input for comparison purpose.

In [7]:
import time

# Measure the latency. It is not accurate using Jupyter Notebook, it is recommended to use standalone python script.
latency = []
with torch.no_grad():
    for i in range(total_samples):
        data = dataset[i]
        inputs = {
            'input_ids':      data[0].to(device).reshape(1, max_seq_length),
            'attention_mask': data[1].to(device).reshape(1, max_seq_length),
            'token_type_ids': data[2].to(device).reshape(1, max_seq_length)
        }
        start = time.time()
        outputs = model(**inputs)
        latency.append(time.time() - start)
print("PyTorch {} Inference time = {} ms".format(device.type, format(sum(latency) * 1000 / len(latency), '.2f')))

PyTorch cuda Inference time = 16.57 ms


## 4. Inference ONNX Model with ONNX Runtime ##

### CUDA and cuDNN Path
onnxruntime-gpu has dependency on [CUDA](https://developer.nvidia.com/cuda-downloads) and [cuDNN](https://developer.nvidia.com/cudnn):

* [onnxruntime-gpu v1.3.0](https://github.com/microsoft/onnxruntime/tree/rel-1.3.0#system-requirements) requires CUDA Runtime 10.1 and CUDNN 7.6.5.
* [onnxruntime-gpu v1.2.0](https://github.com/microsoft/onnxruntime/releases/tag/v1.2.0) requires CUDA Runtime 10.1 and CUDNN 7.6.5.

During installing PyTorch 1.5, we installed cudatoolkit 10.1.243 in this conda environment. That shall be good for onnxruntime-gpu 1.3.0 in Jupyter Notebook.

In [8]:
# Change to True when onnxruntime (like onnxruntime-gpu 1.0.0 ~ 1.1.2) cannot be imported.
add_cuda_path = False

if add_cuda_path:
    # Add path of CUDA 10.0 and CUDNN 7.6 for onnxruntime-gpu 1.0.0 ~ 1.1.2
    cuda_dir = 'D:/NVidia/CUDA/v10.1/bin'
    cudnn_dir = 'D:/NVidia/CUDA/v10.1/bin'
    if not (os.path.exists(cuda_dir) and os.path.exists(cudnn_dir)):
        raise ValueError("Please specify correct path for CUDA and cuDNN. Otherwise onnxruntime cannot be imported.")
    else:
        if cuda_dir == cudnn_dir:
            os.environ["PATH"] = cuda_dir + ';' + os.environ["PATH"]
        else:
            os.environ["PATH"] = cuda_dir + ';' + cudnn_dir + ';' + os.environ["PATH"]

### OpenMP Environment Variable

OpenMP environment variables are optional for GPU inference of standard Bert model. It has little performance impact on Bert model since most nodes are executed in GPU. 

You can find the best setting based on [Performance Test Tool](#Performance-Test-Tool) result in later part of this notebook.

**Attention: Setting environment variables shall be done before importing onnxruntime**. Otherwise, they might not take effect.

In [9]:
# Optional. You can change them according to Performance Test Tool result.
#os.environ["OMP_NUM_THREADS"] = '1'
#os.environ["OMP_WAIT_POLICY"] = 'PASSIVE'

Now we are ready to inference the model with ONNX Runtime.

In [10]:
import psutil
import onnxruntime
import numpy

assert 'CUDAExecutionProvider' in onnxruntime.get_available_providers()
device_name = 'gpu'

sess_options = onnxruntime.SessionOptions()

# Optional: store the optimized graph and view it using Netron to verify that model is fully optimized.
# Note that this will increase session creation time so enable it for debugging only.
sess_options.optimized_model_filepath = os.path.join(output_dir, "optimized_model_{}.onnx".format(device_name))

# Please change the value according to best setting in Performance Test Tool result.
sess_options.intra_op_num_threads=psutil.cpu_count(logical=True)

session = onnxruntime.InferenceSession(export_model_path, sess_options)

latency = []
for i in range(total_samples):
    data = dataset[i]
    # TODO: use IO Binding (see https://github.com/microsoft/onnxruntime/pull/4206) to improve performance.
    ort_inputs = {
        'input_ids':  data[0].cpu().reshape(1, max_seq_length).numpy(),
        'input_mask': data[1].cpu().reshape(1, max_seq_length).numpy(),
        'segment_ids': data[2].cpu().reshape(1, max_seq_length).numpy()
    }
    start = time.time()
    ort_outputs = session.run(None, ort_inputs)
    latency.append(time.time() - start)
    
print("OnnxRuntime {} Inference time = {} ms".format(device_name, format(sum(latency) * 1000 / len(latency), '.2f')))

OnnxRuntime gpu Inference time = 4.43 ms


We can compare the output of PyTorch and ONNX Runtime. We can see some results are not close. It is because ONNX Runtime uses some approximation in CUDA optimization. Based on our evaluation on SQuAD data set, F1 score is on par for models before and after optimization.

In [11]:
print("***** Verifying correctness *****")
for i in range(2):    
    print('PyTorch and ONNX Runtime output {} are close:'.format(i), numpy.allclose(ort_outputs[i], outputs[i].cpu(), rtol=1e-02, atol=1e-02))
    diff = ort_outputs[i] - outputs[i].cpu().numpy()
    max_diff = numpy.max(numpy.abs(diff))
    avg_diff = numpy.average(numpy.abs(diff))
    print(f'maximum_diff={max_diff} average_diff={avg_diff}')

***** Verifying correctness *****
PyTorch and ONNX Runtime output 0 are close: True
maximum_diff=9.499490261077881e-07 average_diff=1.4225952327251434e-07
PyTorch and ONNX Runtime output 1 are close: True
maximum_diff=6.92903995513916e-07 average_diff=1.2441887520253658e-07


### Inference with Actual Sequence Length
Note that ONNX model is exported using dynamic length axis. It is recommended to use actual sequence input without padding instead of fixed length input for best performance. Let's see how it can be applied to this model.

From an example input below, we can see zero padding at the end of each sequence.

In [12]:
# An example input (we can see padding). From attention_mask, we can deduce the actual length.
inputs

{'input_ids': tensor([[  101,  1293,  1242,  2557,  1127,  1226,  1104,  1103,  3613, 16429,
           5235,   136,   102,  3613, 16429,  5988,   170,   107,  1353,  1671,
           1992,  1342,   107,  5235,   117,  1107,  1134,  1473,  3683,  3538,
           1125,   170,  1476,   118,  1248,  2595,  4086,  1714,  1104,  2965,
          15897,  1104,  3613, 16429,   119,  1473,  3683,  3538,  3222,  1149,
           2551,  1168, 23759,  1116,  1121,  1506,  1103, 10280,  2231,  1111,
           1103,  1714, 16355,   119,   102,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,    

The original sequence length is 128. After removing paddings, the sequence length is reduced. Input with smaller sequence length need less computation, thus we can see there is improvement on inference latency. 

In [13]:
import statistics

latency = []
lengths = []
for i in range(total_samples):
    data = dataset[i]
    # Instead of using fixed length (128), we can use actual sequence length (less than 128), which helps to get better performance.
    actual_sequence_length = sum(data[1].numpy())
    lengths.append(actual_sequence_length)
    opt_inputs = {
        'input_ids':  data[0].numpy()[:actual_sequence_length].reshape(1, actual_sequence_length),
        'input_mask': data[1].numpy()[:actual_sequence_length].reshape(1, actual_sequence_length),
        'segment_ids': data[2].numpy()[:actual_sequence_length].reshape(1, actual_sequence_length)
    }
    start = time.time()
    opt_outputs = session.run(None, opt_inputs)
    latency.append(time.time() - start)
print("Average length", statistics.mean(lengths))
print("OnnxRuntime {} Inference time with actual sequence length = {} ms".format(device_name, format(sum(latency) * 1000 / len(latency), '.2f')))

Average length 101
OnnxRuntime gpu Inference time with actual sequence length = 4.23 ms


Let's compare the output and see whether the results are close.

**Note**: Need end-to-end evaluation on performance and accuracy if you use this strategy.

In [14]:
print("***** Comparing results with/without paddings *****")
for i in range(2):
    print('Output {} are close:'.format(i), numpy.allclose(opt_outputs[i], ort_outputs[i][:,:len(opt_outputs[i][0])], rtol=1e-03, atol=1e-03))

***** Comparing results with/without paddings *****
Output 0 are close: True
Output 1 are close: True


## 5. Offline Optimization and Test Tools

It is recommended to try [OnnxRuntime Transformer Model Optimization Tool](https://github.com/microsoft/onnxruntime/tree/master/onnxruntime/python/tools/transformers) on the exported ONNX models. It could help verify whether the model can be fully optimized, and get performance test results.

#### Transformer Optimizer

Although OnnxRuntime could optimize Bert model exported by PyTorch. Sometime, model cannot be fully optimized due to different reasons:
* A new subgraph pattern is generated by new version of export tool, and the pattern is not covered by older version of OnnxRuntime. 
* The exported model uses dynamic axis and this makes it harder for shape inference of the graph. That blocks some optimization to be applied.
* Some optimization is better to be done offline. Like change input tensor type from int64 to int32 to avoid extra Cast nodes, or convert model to float16 to achieve better performance in V100 or T4 GPU.

We have python script **optimizer.py**, which is more flexible in graph pattern matching and model conversion (like float32 to float16). You can also use it to verify whether a Bert model is fully optimized.

In this example, we can see that it introduces optimization that is not provided by onnxruntime: SkipLayerNormalization and bias fusion, which is not fused in OnnxRuntime due to shape inference as mentioned.

It will also tell whether the model is fully optimized or not. If not, that means you might need change the script to fuse some new pattern of subgraph.

Example Usage:
```
from onnxruntime_tools import optimizer
optimized_model = optimizer.optimize_model(export_model_path, model_type='bert', num_heads=12, hidden_size=768)
optimized_model.save_model_to_file(optimized_model_path)
```

You can also use optimizer_cli like the following:

#### Float32 Model
Let us optimize the ONNX model using the script. The first example will output model with float32 to store weights. This is the choice for most GPUs without Tensor Core.

If your GPU (like V100 or T4) has Tensor Core, jump to [Float16 Model](#6.-Model-Optimization-with-Float16) section since that will give you better performance than Float32 model.

In [15]:
optimized_fp32_model_path = './onnx/bert-base-cased-squad_opt_{}_fp32.onnx'.format('gpu' if use_gpu else 'cpu')

!python -m onnxruntime_tools.optimizer_cli --input $export_model_path --output $optimized_fp32_model_path

optimize_by_onnxruntime: Save optimized model by onnxruntime to ./onnx/bert-base-cased-squad_opset11_o1_cpu.onnx
               apply: Fused LayerNormalization count: 25
               apply: Fused Gelu count: 12
               apply: Fused SkipLayerNormalization count: 25
               apply: Fused Attention count: 12
         prune_graph: Graph pruned: 0 inputs, 0 outputs and 5 nodes are removed
               apply: Fused EmbedLayerNormalization(with mask) count: 1
         prune_graph: Graph pruned: 0 inputs, 0 outputs and 12 nodes are removed
         prune_graph: Graph pruned: 0 inputs, 0 outputs and 0 nodes are removed
               apply: Fused BiasGelu count: 12
               apply: Fused SkipLayerNormalization(add bias) count: 24
            optimize: opset verion: 11
  save_model_to_file: Output model to ./onnx/bert-base-cased-squad_opt_gpu_fp32.onnx
get_fused_operator_statistics: Optimized operators:{'EmbedLayerNormalization': 1, 'Attention': 12, 'Gelu': 0, 'FastGelu': 0

#### Optimized Graph
We can open the optimized model using [Netron](https://github.com/lutzroeder/netron) to visualize.

The graph is like the following:
<img src='images/optimized_bert_gpu.png'>

Sometime, optimized graph is slightly different. For example, FastGelu is replaced by BiasGelu for CPU inference; When the option --input_int32 is used, Cast nodes for inputs are removed.

In [16]:
import netron

# change it to True if want to view the optimized model in browser
enable_netron = False
if enable_netron:
    # If you encounter error "access a socket in a way forbidden by its access permissions", install Netron as standalone application instead.
    netron.start(optimized_fp32_model_path)

### Performance Test Tool

The following will create 1000 random inputs of batch_size 1 and sequence length 128, then measure the average latency and throughput numbers.

Note that the test uses fixed sequence length. If you use [dynamic sequence length](#Inference-with-Actual-Sequence-Length), actual performance depends on the distribution of sequence length.

**Attention**: Latency numbers from Jupyter Notebook are not accurate. See [Attional Info](#7.-Additional-Info) for more info.

In [17]:
GPU_OPTION = '--use_gpu' if use_gpu else ''

!python -m onnxruntime_tools.transformers.bert_perf_test --model $optimized_fp32_model_path --batch_size 1 --sequence_length 128 --samples 1000 --test_times 1 --inclusive --all $GPU_OPTION

test setting TestSetting(batch_size=1, sequence_length=128, test_cases=1000, test_times=1, contiguous=None, use_gpu=True, warmup=True, omp_num_threads=None, omp_wait_policy=None, intra_op_num_threads=None, seed=3, verbose=False, inclusive=False, extra_latency=True)
Generating 1000 samples for batch_size=1 sequence_length=128
Running test: model=bert-base-cased-squad_opt_gpu_fp32.onnx,graph_optimization_level=ENABLE_ALL,intra_op_num_threads=0,OMP_NUM_THREADS=,OMP_WAIT_POLICY=,batch_size=1,sequence_length=128,test_cases=1000,test_times=1,contiguous=None,use_gpu=True,warmup=True
Average latency = 4.92 ms, Throughput = 203.24 QPS
Running test: model=bert-base-cased-squad_opt_gpu_fp32.onnx,graph_optimization_level=ENABLE_ALL,intra_op_num_threads=1,OMP_NUM_THREADS=12,OMP_WAIT_POLICY=PASSIVE,batch_size=1,sequence_length=128,test_cases=1000,test_times=1,contiguous=None,use_gpu=True,warmup=True
Average latency = 4.90 ms, Throughput = 203.88 QPS
Running test: model=bert-base-cased-squad_opt_gpu_

Let's load the summary file and take a look. Note that blank value in OMP_NUM_THREADS or OMP_WAIT_POLICY means the environment variable does not exist.

In [18]:
import os
import glob     
import pandas
latest_result_file = max(glob.glob("./onnx/perf_results_GPU_B1_S128_*.txt"), key=os.path.getmtime)
result_data = pandas.read_table(latest_result_file, converters={'OMP_NUM_THREADS': str, 'OMP_WAIT_POLICY':str})
print("Float32 model perf results from", latest_result_file)
# Remove some columns that have same values for all rows.
columns_to_remove = ['model', 'graph_optimization_level', 'batch_size', 'sequence_length', 'test_cases', 'test_times', 'use_gpu']
result_data.drop(columns_to_remove, axis=1, inplace=True)
result_data

Float32 model perf results from ./onnx/perf_results_GPU_B1_S128_20200617-232134.txt


,Latency(ms),Latency_P50,Latency_P75,Latency_P90,Latency_P95,Latency_P99,Throughput(QPS),intra_op_num_threads,OMP_NUM_THREADS,OMP_WAIT_POLICY,contiguous,warmup
0,4.82,4.53,4.57,5.15,7.25,8.75,207.33,1,12,ACTIVE,None,True
1,4.88,4.54,4.58,6.47,7.13,8.68,204.90,12,12,PASSIVE,None,True
2,4.90,4.54,4.57,6.16,7.64,8.82,203.88,1,12,PASSIVE,None,True
3,4.91,4.55,4.59,6.70,7.43,8.78,203.55,12,12,ACTIVE,None,True
4,4.92,4.57,4.60,6.50,7.82,8.90,203.24,0,,,None,True
5,4.93,4.55,4.59,6.66,7.57,8.80,202.92,12,1,PASSIVE,None,True
6,5.07,4.56,4.61,7.19,8.11,9.01,197.16,12,1,ACTIVE,None,True


From above result, we can see that latency is very close for different settings. The default setting (intra_op_num_threads=0, OMP_NUM_THREADS and OMP_WAIT_POLICY does not exist) performs the best. 

### Model Results Comparison Tool

When a BERT model is optimized, some approximation is used in calculation. If your BERT model has three inputs, a script compare_bert_results.py can be used to do a quick verification. The tool will generate some fake input data, and compare the inference outputs of the original and optimized models. If outputs are all close, it is safe to use the optimized model.

For GPU inference, the absolute or relative difference is larger than those numbers of CPU inference. Note that slight difference in output will not impact final result. We did end-to-end evaluation using SQuAD data set using a fine-tuned squad model, and F1 score is almost the same before/after optimization.

In [19]:
!python -m onnxruntime_tools.transformers.compare_bert_results --baseline_model $export_model_path --optimized_model $optimized_fp32_model_path --batch_size 1 --sequence_length 128 --samples 100 --rtol 0.01 --atol 0.01 $GPU_OPTION

100% passed for 100 random inputs given thresholds (rtol=0.01, atol=0.01).
maximum absolute difference=1.9222497940063477e-06
maximum relative difference=0.05027933046221733


## 6. Model Optimization with Float16

The optimizer.py script have an option **--float16** to convert model to use float16 to store weights. After the conversion, it could be faster to run in GPU with tensor cores like V100 or T4.

Let's run tools to measure the performance on V100. The results show significant performance improvement: latency is about 3.4 ms for float32 model, and 1.8 ms for float16 model.

In [20]:
optimized_fp16_model_path = './onnx/bert-base-cased-squad_opt_{}_fp16.onnx'.format('gpu' if use_gpu else 'cpu')
!python -m onnxruntime_tools.optimizer_cli --input $export_model_path --output $optimized_fp16_model_path --float16

optimize_by_onnxruntime: Save optimized model by onnxruntime to ./onnx/bert-base-cased-squad_opset11_o1_cpu.onnx
               apply: Fused LayerNormalization count: 25
               apply: Fused Gelu count: 12
               apply: Fused SkipLayerNormalization count: 25
               apply: Fused Attention count: 12
         prune_graph: Graph pruned: 0 inputs, 0 outputs and 5 nodes are removed
               apply: Fused EmbedLayerNormalization(with mask) count: 1
         prune_graph: Graph pruned: 0 inputs, 0 outputs and 12 nodes are removed
         prune_graph: Graph pruned: 0 inputs, 0 outputs and 0 nodes are removed
               apply: Fused BiasGelu count: 12
               apply: Fused SkipLayerNormalization(add bias) count: 24
            optimize: opset verion: 11
  save_model_to_file: Output model to ./onnx/bert-base-cased-squad_opt_gpu_fp16.onnx
get_fused_operator_statistics: Optimized operators:{'EmbedLayerNormalization': 1, 'Attention': 12, 'Gelu': 0, 'FastGelu': 0

In [21]:
GPU_OPTION = '--use_gpu' if use_gpu else ''
!python -m onnxruntime_tools.transformers.bert_perf_test --model $optimized_fp16_model_path --batch_size 1 --sequence_length 128 --samples 1000 --test_times 1 --inclusive --all $GPU_OPTION

test setting TestSetting(batch_size=1, sequence_length=128, test_cases=1000, test_times=1, contiguous=None, use_gpu=True, warmup=True, omp_num_threads=None, omp_wait_policy=None, intra_op_num_threads=None, seed=3, verbose=False, inclusive=False, extra_latency=True)
Generating 1000 samples for batch_size=1 sequence_length=128
Running test: model=bert-base-cased-squad_opt_gpu_fp16.onnx,graph_optimization_level=ENABLE_ALL,intra_op_num_threads=0,OMP_NUM_THREADS=,OMP_WAIT_POLICY=,batch_size=1,sequence_length=128,test_cases=1000,test_times=1,contiguous=None,use_gpu=True,warmup=True
Average latency = 3.01 ms, Throughput = 331.90 QPS
Running test: model=bert-base-cased-squad_opt_gpu_fp16.onnx,graph_optimization_level=ENABLE_ALL,intra_op_num_threads=1,OMP_NUM_THREADS=12,OMP_WAIT_POLICY=PASSIVE,batch_size=1,sequence_length=128,test_cases=1000,test_times=1,contiguous=None,use_gpu=True,warmup=True
Average latency = 3.12 ms, Throughput = 320.00 QPS
Running test: model=bert-base-cased-squad_opt_gpu_

In [22]:
import os
import glob     
import pandas
latest_result_file = max(glob.glob("./onnx/perf_results_GPU_B1_S128_*.txt"), key=os.path.getmtime)
result_data = pandas.read_table(latest_result_file, converters={'OMP_NUM_THREADS': str, 'OMP_WAIT_POLICY':str})
print("Float32 model perf results from", latest_result_file)
# Remove some columns that have same values for all rows.
columns_to_remove = ['model', 'graph_optimization_level', 'batch_size', 'sequence_length', 'test_cases', 'test_times', 'use_gpu']
result_data.drop(columns_to_remove, axis=1, inplace=True)
result_data

Float32 model perf results from ./onnx/perf_results_GPU_B1_S128_20200617-232234.txt


,Latency(ms),Latency_P50,Latency_P75,Latency_P90,Latency_P95,Latency_P99,Throughput(QPS),intra_op_num_threads,OMP_NUM_THREADS,OMP_WAIT_POLICY,contiguous,warmup
0,3.01,2.79,2.81,2.86,5.08,7.16,332.53,1,12,ACTIVE,None,True
1,3.01,2.80,2.81,2.88,4.52,7.05,331.90,0,,,None,True
2,3.01,2.78,2.80,2.92,5.01,7.02,331.72,12,12,ACTIVE,None,True
3,3.02,2.79,2.80,2.85,6.34,7.04,331.39,12,1,ACTIVE,None,True
4,3.04,2.80,2.82,2.93,5.56,7.08,329.32,12,12,PASSIVE,None,True
5,3.04,2.79,2.81,2.92,6.37,7.08,328.67,12,1,PASSIVE,None,True
6,3.12,2.79,2.82,2.96,6.66,7.20,320.00,1,12,PASSIVE,None,True


### Throughput Tuning

Some application need best throughput under some constraint on latency. This can be done by testing performance of different batch sizes. The tool could help on this.

Here is an example that check the performance of multiple batch sizes (1, 2, 4, 8, 16, 32 and 64) using default settings.

In [23]:
GPU_OPTION = '--use_gpu' if use_gpu else ''
THREAD_SETTING = '--intra_op_num_threads 1 --omp_num_threads {} --omp_wait_policy ACTIVE'.format(psutil.cpu_count(logical=True))
!python -m onnxruntime_tools.transformers.bert_perf_test --model $optimized_fp16_model_path --batch_size 1 2 4 8 16 32 64 --sequence_length 128 --samples 1000 --test_times 1 --inclusive $THREAD_SETTING $GPU_OPTION

test setting TestSetting(batch_size=32, sequence_length=128, test_cases=1000, test_times=1, contiguous=None, use_gpu=True, warmup=True, omp_num_threads=12, omp_wait_policy='ACTIVE', intra_op_num_threads=1, seed=3, verbose=False, inclusive=False, extra_latency=True)
Generating 1000 samples for batch_size=32 sequence_length=128
Running test: model=bert-base-cased-squad_opt_gpu_fp16.onnx,graph_optimization_level=ENABLE_ALL,intra_op_num_threads=1,OMP_NUM_THREADS=12,OMP_WAIT_POLICY=ACTIVE,batch_size=32,sequence_length=128,test_cases=1000,test_times=1,contiguous=None,use_gpu=True,warmup=True
Average latency = 16.17 ms, Throughput = 1979.41 QPS
test setting TestSetting(batch_size=1, sequence_length=128, test_cases=1000, test_times=1, contiguous=None, use_gpu=True, warmup=True, omp_num_threads=12, omp_wait_policy='ACTIVE', intra_op_num_threads=1, seed=3, verbose=False, inclusive=False, extra_latency=True)
Generating 1000 samples for batch_size=1 sequence_length=128
Running test: model=bert-bas

In [26]:
import os
import glob     
import pandas
latest_result_file = max(glob.glob("./onnx/perf_results_*.txt"), key=os.path.getmtime)
result_data = pandas.read_table(latest_result_file, converters={'OMP_NUM_THREADS': str, 'OMP_WAIT_POLICY':str})
print("Float16 model summary from", latest_result_file)
columns_to_remove = ['model', 'graph_optimization_level', 'test_cases', 'test_times', 'use_gpu', 'warmup', 'sequence_length']
columns_to_remove.extend(['intra_op_num_threads', 'OMP_NUM_THREADS', 'OMP_WAIT_POLICY', 'contiguous'])
result_data.drop(columns_to_remove, axis=1, inplace=True)
result_data

Float16 model summary from ./onnx/perf_results_GPU_B1-2-4-8-16-32-64_S128_20200617-232401.txt


,Latency(ms),Latency_P50,Latency_P75,Latency_P90,Latency_P95,Latency_P99,Throughput(QPS),batch_size
0,3.00,2.79,2.81,2.86,4.37,7.08,333.83,1
1,3.59,3.33,3.35,3.42,6.60,7.54,557.32,2
2,4.32,3.98,4.01,4.64,7.23,8.11,926.92,4
3,6.32,5.94,5.97,7.61,8.96,10.12,1266.63,8
4,9.60,9.22,9.25,11.32,12.33,13.34,1666.05,16
5,16.17,15.80,15.90,17.38,18.80,19.93,1979.41,32
6,29.26,28.89,29.01,30.63,32.53,33.28,2187.15,64


## 7. Additional Info

Note that running Jupyter Notebook has significant impact on performance result. You can close Jupyter Notebook and other applications, then run the performance test in a console to get more accurate performance numbers.

We have a [benchmark script](https://github.com/microsoft/onnxruntime/blob/master/onnxruntime/python/tools/transformers/run_benchmark.sh). It is recommended to use it measure inference speed of OnnxRuntime.

[OnnxRuntime C API](https://github.com/microsoft/onnxruntime/blob/master/docs/C_API.md) could get slightly better performance than python API. If you use C API in inference, you can use OnnxRuntime_Perf_Test.exe built from source to measure performance instead.

Here is the machine configuration that generated the above results. You might get slower or faster result according to your hardware.

In [27]:
!{sys.executable} -m onnxruntime_tools.transformers.machine_info --silent

{
  "gpu": {
    "driver_version": "440.64.00",
    "devices": [
      {
        "memory_total": 16945512448,
        "memory_available": 14110883840,
        "name": "Tesla V100-PCIE-16GB"
      },
      {
        "memory_total": 16945512448,
        "memory_available": 16932601856,
        "name": "Tesla V100-PCIE-16GB"
      }
    ]
  },
  "cpu": {
    "brand": "Intel(R) Xeon(R) CPU E5-2690 v4 @ 2.60GHz",
    "cores": 12,
    "logical_cores": 12,
    "hz": "2.5940 GHz",
    "l2_cache": "256 KB",
    "l3_cache": "35840 KB",
    "processor": "x86_64"
  },
  "memory": {
    "total": 236645588992,
    "available": 222567559168
  },
  "python": "3.7.7.final.0 (64 bit)",
  "os": "Linux-4.15.0-1089-azure-x86_64-with-debian-stretch-sid",
  "onnxruntime": {
    "version": "1.3.0",
    "support_gpu": true
  },
  "pytorch": {
    "version": "1.5.0",
    "support_gpu": true
  },
  "tensorflow": null
}
